In [515]:
#IMPORT
import random
import hashlib
from itertools import chain
import hashlib
import binascii
import sys
import pickle

In [516]:
#PARAMS
p = 127
Fp = GF(p)
n = 128
k = 76
w = 9
r = n-k
t = 16
z = 7
Fz = GF(z)
lamb = 128
lev_leaf = log(t,2)
Fp_set = Set(Fp)
Fp_star = Fp_set.difference([0])

In [517]:
#FUNZIONI DI UTILITA

def generate_random_seed():
    set_random_seed()
    return initial_seed()

def set_seed(seed):
    set_random_seed(seed)
    return initial_seed()

def hash_and_get_seed(input):
    hash = hashlib.sha256(str(input).encode()).digest()
    seed = int.from_bytes(hash, 'big')
    return seed

def hexhash(input):
    return hashlib.sha256(input.encode()).hexdigest()

def convert_int_to_hash_string(input):
    return input.to_bytes((input.bit_length() + 7) // 8, 'big').hex()

def expand_seed(seed_init):
    seed_str = str(seed_init)
    seeds = hashlib.sha256(seed_str.encode()).digest()
    
    seed_1 = seeds[:16] #i primi 16 byte
    seed_2 = seeds[16:] #gli ultimi 16 byte
    
    seed_1 = int.from_bytes(seed_1, 'big')
    seed_2 = int.from_bytes(seed_2, 'big')
    
    return seed_1, seed_2

#Creo Generatore g del Campo E
def get_generator(p, z, Fp):
    assert is_prime(p),"p is not prime!"
    assert (p-1)%z == 0, "z does not divide p-1!"
    alpha = Fp.primitive_element()
    #non ricordo perchè exp e g sono scelti secondo questa logica
    exp = (p-1)/z
    g = alpha**exp
    return g

def create_random_vector(domain, n_elems, seed):
    set_seed(seed)
    return random_vector(domain, n_elems)

def create_random_matrix(domain, n_rows, n_columns, seed):
    set_seed(seed)
    M = random_matrix(domain, n_rows, n_columns)
    return M

#Creo vettore errore con elementi in E e lunghezza V 
def create_vEn(g, n, Fz, Fp, seed=0, eta=[]):
    e = vector(Fp, n)
    if len(eta)==0:
        eta = create_random_vector(Fz, n, seed)
    for i in range(0, n):#range check
        e[i] = (g**eta[i])
    return e

def create_vector_over_Fp_star(Fp_star, t, seed):
    set_random_seed(seed)
    return [Fp_star.random_element() for i in range(t)]

def generate_vector_fixed_hamming_w(w, t, seed):
    random.seed(seed)
    vector = [0] * t
    indices = random.sample(range(t), w)
    for index in indices:
        vector[index] = 1
    return vector

def component_wise_multiply(v1, v2):
    if len(v1) != len(v2):
        raise ValueError("I vettori devono avere la stessa dimensione")
    result = vector(GF(p), len(v1))  # Crea un nuovo vettore per i risultati
    for i in range(len(v1)):
        result[i] = v1[i] * v2[i]  # Moltiplica gli elementi corrispondenti
    
    return result

def convert_object_to_binary_string(input):
    # Serializza l'oggetto in un formato binario
    binary_data = pickle.dumps(input)
    # Converte la stringa binaria in una rappresentazione di stringa binaria
    binary_string = ''.join(format(byte, '08b') for byte in binary_data)
    return binary_string

def get_object_from_binary_string(binary_string):
    # Converte la stringa binaria in una rappresentazione binaria dei byte
    binary_data = bytes(int(binary_string[i:i+8], 2) for i in range(0, len(binary_string), 8))
    # Deserializza i dati binari in un oggetto
    output = pickle.loads(binary_data)
	


In [518]:
#SEED TREE

class SeedTree:
    def __init__(self, depth, salt):
        if(depth>0):
            self.seed_m = None
            self.depth = depth
            self.salt = str(salt)
            self.tree = None
            self.full = False
            self.seed_path = None
        else:
            raise Exception('Il numero delle foglie deve essere una porenza di 2')

    def make_tree(self, seed_m):
        if self.full:
            raise Exception('Albero gia creato')
        else:
            self.seed_m = seed_m
            #BISOGNA AGGIUNGERE CONTROLLI?
            self.tree = []
            self.__make_tree([self.seed_m], 0)
            self.full = True
        
    def get_leaves(self):
        if self.full:
            return self.tree[self.depth]
        else:
            raise Exception('Albero non creato')

    def compute_level(self, parents, level):
        return self.__create_new_level(parents, level) 

    def get_seed_path(self):
        if self.full and self.seed_path is not None:
            return self.seed_path

    def compute_seed_path(self, list_nodes):
        if all(self.full or 1 <= node <= 2**self.depth for node in list_nodes):
            self.seed_path={}
            self.__compute_seed_path(self.depth, list_nodes)
        else:
            raise Exception('Albero non creato o foglia non valida')
        
    def get_required_leaves_from_seed_path(self, seed_path, leaves_indexes):
        leaves = self.__get_required_leaves_from_seed_path(seed_path)
        if list(leaves.keys())==leaves_indexes:
            return leaves
        else:
            raise Exception('Seed Path non corretto')
    

    def __make_tree(self, list_nodes, lev):
        if(lev<=self.depth):
            self.tree.append(list_nodes)
            list_children = self.__create_new_level(list_nodes, lev)
            self.__make_tree(list_children, lev+1)

    def __create_new_level(self, parents, level):
        children = []
        if level == 0:
            sx, dx = self.__compute_children_value(parents[0], '')
            children.append(sx)
            children.append(dx)
        else:
            base = 2**level
            for i in range(0, len(parents)):
                sx, dx = self.__compute_children_value(parents[i], base+i)
                children.append(sx)
                children.append(dx)
        return children
    
    def __recompute_new_level(self, parents, level):
        children = {}
        base = 2**level
        for i in parents.keys():
            sx, dx = self.__compute_children_value(parents[i], base+i)
            children[i*2]=sx
            children[(i*2)+1]=dx
        return children
        
    def __compute_children_value(self, seed_parent, index_parent):
        nodes = str(seed_parent) + self.salt + str(index_parent)
        sx, dx = expand_seed(int(nodes))  
        return sx, dx
    
    def __compute_seed_path(self, depth, list_index):
        if(depth > 1):
            list_index_next_level= list(filter(lambda x: (x % 2 == 0 and x + 1 in list_index) or (x % 2 != 0 and x - 1 in list_index), list_index)) 
            list_index = list(filter(lambda x: x not in list_index_next_level, list_index)) 
            list_parent = sorted(list(set(map(lambda elem: (elem // 2), list_index_next_level)))) 
            if len(list_index)>0:
                self.seed_path[depth] = dict(map(lambda x: (x, self.tree[depth][x]), list_index)) 
            if len(list_parent)>0:
                self.__compute_seed_path(depth-1, list_parent) 
    
    def __get_required_leaves_from_seed_path(self, seed_path):
        if len(seed_path.keys())>1:
            index = min(seed_path.keys())
            current_nodes=seed_path.pop(index)
            recomputed_children = self.__recompute_new_level(current_nodes, index)
            new_index=min(seed_path.keys())
            children = seed_path[new_index]
            merged_children = {**children, **recomputed_children} 
            seed_path[new_index] = dict(sorted(merged_children.items()))
            return self.__get_required_leaves_from_seed_path(seed_path)
        else:
            index, current_nodes = seed_path.popitem()
            return current_nodes


            

In [519]:
#MERKLE TREE

class MerkleTree:
    def __init__(self, n_leaves):
        if(n_leaves%2==0):
            self.n_leaves = n_leaves
            self.depth = log(n_leaves, 2) 
            self.tree =  None
            self.merkle_proof =  None
            self.full = False
        else:
            raise Exception('Il numero delle foglie deve essere una porenza di 2')

    #METODI PUBBLICI 
    def make_tree(self, leaves):
        if isinstance(leaves, list) and len(leaves) == self.n_leaves and not(self.full): 
            self.tree = []
            self.tree.append(leaves)  
            self.__make_tree(leaves, self.depth-1) 
            self.full = True
        else:
            raise Exception('Il numero delle foglie deve essere pari a {}'.format(self.n_leaves))
       
    def get_root(self):
        if self.full:
            return self.tree[0][0]
        else:
            raise Exception('Albero non creato')

    def get_merkle_proof(self):
        if self.full and self.merkle_proof is not None:
            return self.merkle_proof

    def compute_proof(self, list_nodes):
        if all(self.full or 1 <= node <= 2**self.depth for node in list_nodes):
            self.merkle_proof={}
            list_leaves = sorted(list(map(lambda x: x+1 if x % 2 == 0 else x-1, list_nodes))) 
            self.__compute_merkle_proof(self.depth, list_leaves)
            list_leaves = list(filter(lambda x: x not in list_nodes, list_leaves)) 
            self.merkle_proof[self.depth] = dict(map(lambda x: (x, self.tree[self.depth][x]), list_leaves)) 
            #self.merkle_proof = sorted(self.merkle_proof.items())
            self.merkle_proof = {k: self.merkle_proof[k] for k in sorted(self.merkle_proof, key=lambda x: int(x))}
        else:
            raise Exception('Albero non creato o foglia non valida')

    def recompute_root(self, merkle_proof, leaves):
        if len(leaves)<self.n_leaves:
            return self.__recompute_root(merkle_proof, leaves)
    
    #METODI PRIVATI

    def __make_tree(self, list_nodes, depth): 
        if(depth>=0): 
            list_parent = self.__create_new_level(list_nodes) 
            self.tree.insert(0, list_parent)
            self.__make_tree(list_parent, depth-1) 

    def __create_new_level(self, child_nodes):
        parents = [] 
        for i in range(0, len(child_nodes), 2): 
            new_node = self.__compute_father_value(child_nodes[i], child_nodes[i+1]) 
            parents.append(new_node)
        return parents
        
    def __compute_father_value(self, sx, dx):
        return hexhash(sx+dx)
    
    def __compute_merkle_proof(self, depth, list_index):
        if(depth > 1): 
            list_parent = sorted(list(set(map(lambda elem: ((elem // 2) + 1) if (elem // 2) % 2 == 0 else (elem // 2) - 1, list_index))))  
            parent_proof = list(filter(lambda x: x*2 not in list_index and (x*2)+1 not in list_index, list_parent))
            if len(parent_proof)>0:
                self.merkle_proof[depth-1] = dict(map(lambda x: (x, self.tree[depth-1][x]), parent_proof)) 
            self.__compute_merkle_proof(depth-1, list_parent) 

    def __recompute_root(self, merkle_proof, nodes):
        if len(nodes)==2:
            return self.__compute_father_value(nodes[0],nodes[1])
        else:
            if len(merkle_proof.keys())>0:
                index = max(merkle_proof.keys())
                current_level = merkle_proof.pop(index)
                merged_nodes = {**current_level, **nodes}
                merged_nodes = dict(sorted(merged_nodes.items()))
            else:
                merged_nodes=nodes
            new_nodes = {}
            for key in merged_nodes:
                new_key = key//2
                if new_key in new_nodes.keys():
                    new_nodes[new_key] = self.__compute_father_value(new_nodes[new_key],merged_nodes[key])
                else:
                    new_nodes[new_key] = merged_nodes[key]
            return self.__recompute_root(merkle_proof, new_nodes)

In [520]:
#GENERAZIONE CHIAVI
def key_gen():
    seed_sk = generate_random_seed() 
    seed_e , seed_pk = expand_seed(seed_sk)
    V = create_random_matrix(Fp, k, r, seed_pk) 
    g = get_generator(p, z, Fp)
    e = create_vEn(g, n, Fz, Fp, seed=seed_e) #e=g^η
    s = e[0:r] + e[r:n]*V
    return seed_sk, seed_pk, s


In [521]:
#SIGNER
class Signer:
    def __init__(self, seed_sk):
        seed_e , seed_pk = expand_seed(seed_sk) 
        self.V = create_random_matrix(Fp, k, r, seed_pk)
        self.eta = create_random_vector(Fz, n, seed_e)

    def sign_msg(self, msg, lev_leaf):
        self.__compute_commitment(lev_leaf)
        self.__generate_first_challenge(msg)
        self.__generate_first_response()
        self.__generate_second_challenge()
        self.__generate_second_response()
        return self.__sign()

    def __compute_commitment(self, lev_leaf):
        seed_m = generate_random_seed() 
        seed_salt = generate_random_seed()
        self.salt = hash_and_get_seed(seed_salt)
        self.seed_tree = SeedTree(lev_leaf, self.salt)
        self.seed_tree.make_tree(seed_m)
        self.seeds = self.seed_tree.get_leaves()
        list_cmt0 = [] 
        list_cmt1 = ""
        g = get_generator(p, z, Fp)
        for i in range(0, t):
            seed_ui, seed_ei = expand_seed(self.seeds[i])
            eta_i = create_random_vector(Fz, n, seed_ei)
            sigma_i = self.eta-eta_i #sigma_i ∈ Fz^ns
            v = create_vEn(g, n, Fz, Fp, eta=sigma_i)
            ui = create_random_vector(Fp, n, seed_ui)
            u = component_wise_multiply(v, ui)
            st = u[0:r] + u[r:n]*self.V
            cmt0_i = hexhash(str(st) + str(sigma_i) + str(self.salt) + str(i))
            cmt1_i = hexhash(str(self.seeds[i]) + str(self.salt) + str(i))
            list_cmt0.append(cmt0_i) 
            list_cmt1=list_cmt1+cmt1_i
        #commitment 0
        self.merkle_tree = MerkleTree(t)
        self.merkle_tree.make_tree(list_cmt0)
        c0 = self.merkle_tree.get_root()
        #commitment 1
        #input_c1 = "".join(map(str, list_cmt1))
        c1 = hexhash(list_cmt1) 
        #hash dei due commitment
        self.c01 = hexhash(c0+c1)

    def __generate_first_challenge(self, msg):
        hash_salt = convert_int_to_hash_string(self.salt)
        input_ch1 = hexhash(msg)+self.c01+hash_salt
        self.seed_ch1 = hash_and_get_seed(input_ch1)
        self.ch1 = create_vector_over_Fp_star(Fp_star, t, self.seed_ch1) 

    def __generate_first_response(self):
        #list_hash_yi=[]
        list_yi=""
        g = get_generator(p, z, Fp)
        for seed_i, ch_i_1 in zip(self.seeds, self.ch1): #t round: rispota yi dipende da seed_i e ch_i_1
            seed_ui, seed_ei = expand_seed(seed_i) 
            ei = create_vEn(g, n, Fz, Fp, seed=seed_ei)
            ui = create_random_vector(Fp, n, seed_ui)
            yi = ui+(ch_i_1)*ei #calcolo risposta i-esima corrispondente all'elemento i-esmino del vettore ch1 
            list_yi=list_yi+str(yi)
            #hi = hexhash(str(yi))
            #list_hash_yi.append(hi)
        self.h = hexhash(list_yi)

    def __generate_second_challenge(self):#, seed_tree):
        input_ch2 = "".join(self.h)+str(self.seed_ch1)
        self.seed_ch2 = hash_and_get_seed(input_ch2)
        self.ch2 = generate_vector_fixed_hamming_w(w, t, self.seed_ch2)

    def __generate_second_response(self):#CONTROLLARE ORDINE
        index_1, index_0 = [], []
        #questi due vettori sono di lunghezza t-w ovvero i round corrisponenti agli indici in cui il vettore ch2=0
        self.rsp_ch0 = [] # Fp^n x Fz^m
        self.rsp_ch1 = [] # {0,1}^lambda
        g = get_generator(p, z, Fp)
        #
        i = 0
        for seed_i, ch_i_2 in zip(self.seeds, self.ch2):
            if ch_i_2 == 1:#Challenge2 = 1
                index_1.append(i)  
                
            else: #Creo la risposta nel caso challenge2 = 0
                index_0.append(i)

                seed_ui, seed_ei = expand_seed(seed_i)
                #ricalcolo sigma i
                eta_i = create_random_vector(Fz, n, seed_ei)
                sigma_i = self.eta-eta_i #sigma_i ∈ Fz^ns
                #ricalcolo yi
                ei = create_vEn(g, n, Fz, Fp, seed_ei)
                ui = create_random_vector(Fp, n, seed_ui) 
                yi = ui+(self.ch1[i])*ei
                
                #risposta challenge 0
                self.rsp_ch0.append([yi, sigma_i]) 

                #risposta challenge 1
                cmt1_i = hexhash( str(seed_i) + str(self.salt) + str(i) )
                
                self.rsp_ch1.append(cmt1_i)

            i = i+1
            
        self.seed_tree.compute_seed_path(index_1)
        self.seed_path = self.seed_tree.get_seed_path()
        self.merkle_tree.compute_proof(index_0)
        self.merkle_proof = self.merkle_tree.get_merkle_proof()

    def __sign(self):
        signature = {}
        signature["salt"] = self.salt
        signature["c01"] = self.c01
        signature["ch2"] = self.seed_ch2
        signature["seed_path"] = self.seed_path
        signature["merkle_proof"] = self.merkle_proof
        signature["rsp_ch0"] = self.rsp_ch0
        signature["rsp_ch1"] = self.rsp_ch1
        #binary_signature = convert_object_to_binary_string(signature)
        #return binary_signature
        return signature

In [522]:
#VERIFIER
class Verifier:

    def check_sign(self, seed_pk, s, msg, sign):
        self.msg = msg
        self.V = create_random_matrix(Fp, k, r, seed_pk)
        self.s = s
        self.__retrieve_sign(sign)
        self.__retrive_challenges()
        return self.__recompute_and_verify_responses()
    

    def __retrieve_sign(self, signature):
        #signature = get_object_from_binary_string(sign)
        self.salt = signature["salt"]
        self.c01 = signature["c01"]
        self.seed_ch2 = signature["ch2"] 
        self.seed_path = signature["seed_path"]
        self.merkle_proof = signature["merkle_proof"]
        self.rsp_ch0 = signature["rsp_ch0"]
        self.rsp_ch1 = signature["rsp_ch1"]
        
    def __retrive_challenges(self):
        hash_salt = convert_int_to_hash_string(self.salt)
        hash_msg = hexhash(self.msg)
        #challenge1
        input_ch1 = hash_msg+self.c01+hash_salt
        self.seed_ch1 = hash_and_get_seed(input_ch1)
        self.ch1 = create_vector_over_Fp_star(Fp_star, t, self.seed_ch1)
        #challenge2
        self.ch2 = generate_vector_fixed_hamming_w(w, t, self.seed_ch2)
        #recupero indici
        #self.index_0 = []
        #self.index_1 = []
        #[self.index_1.append(i) if x == 1 else self.index_0.append(i) for i, x in enumerate(self.ch2)]
        index_1 = []
        for i, x in enumerate(self.ch2):
            if x == 1: index_1.append(i)
        seed_tree = SeedTree(t, self.salt)
        #i seeds servono solo quando ch_2 = 1 
        self.seeds = seed_tree.get_required_leaves_from_seed_path(self.seed_path, index_1)
    
    def __recompute_and_verify_responses(self):
        g = get_generator(p, z, Fp)
        c0={}
        list_cmt1=""
        list_y=""
        j=0
        for i, x in enumerate(self.ch2):
            if x ==1:
                seed_ui, seed_ei = expand_seed(self.seeds[i]) 
                #RICALCOLO RISPOSTA CHALLENGE 1
                ei = create_vEn(g, n, Fz, Fp, seed=seed_ei)
                ui = create_random_vector(Fp, n, seed_ui)   
                yi = ui+(self.ch1[i])*ei
                list_y=list_y+str(yi)
                #RICALCOLO COMMITMENT 1
                #cmt1_i = hexhash(ui, ei, salt, i)
                cmt1_i = hexhash(str(self.seeds[i]) + str(self.salt) + str(i))
                list_cmt1=list_cmt1+cmt1_i
                #hi = hexhash(str(yi))
            else:
                [yi, sigma_i]=self.rsp_ch0[j]
                list_y=list_y+str(yi)
                #verificare sigma_i appartenga a G

                #RICALCOLO SIGMA TILDE 
                v = create_vEn(g, n, Fz, Fp, eta=sigma_i)
                yi_ = component_wise_multiply(v, yi)

                st = yi_[0:r] + yi_[r:n]*self.V - self.ch1[i]*self.s

                #RICALCOLO COMMITMENT 0 
                cmt0_i = hexhash(str(st)+str(sigma_i)+str(self.salt)+str(i))
                c0[i]=cmt0_i

                #ESTRAGGO COMMITMENT 1
                cmt1_i = self.rsp_ch1[j]
                list_cmt1=list_cmt1+cmt1_i
                j = j+1

        merkle_tree = MerkleTree(t)
        
        recomputed_root = merkle_tree.recompute_root(self.merkle_proof, c0)
        recomputed_c1 = hexhash(list_cmt1) 
        
        recomputed_c01 = hexhash(recomputed_root+recomputed_c1)
        recomputed_h = hexhash(list_y)
        input_recomputed_ch2 = "".join(recomputed_h)+str(self.seed_ch1)
        recomputed_ch2 = hash_and_get_seed(input_recomputed_ch2)
        
        return recomputed_c01 == self.c01 and recomputed_ch2 == self.seed_ch2
            
        #h = hexhash(h1,...,hn)
        #c0 = hexhash(c01,...,c0t)
        #VerifyMerkleProof   

    


In [523]:
seed_sk, seed_pk, s = key_gen()
signer = Signer(seed_sk)
signature = signer.sign_msg("buonasera", log(t,2))
verifier = Verifier()
esito = verifier.check_sign(seed_pk, s, "buonasera", signature)
print(esito)



True
